# 연결 리스트 개요
연결 리스트란 컴퓨터과학에서 배열과 함께 **가장 기본이 되는 대표적인 선형 자료구조 중 하나** 로  
다양한 추상 자료형(ADT: Abstract Data Type) 구현의 기반이 됨
>- 동적으로 새로운 노드를 삽입하거나 삭제하기 간편
    - 배열과는 달리 특정 인덱스에 접근하기 위해서 전체를 순서대로 읽어야 하므로 O(n)이 소요됨 (배열은 O(1))
    - 그래도 시작 또는 끝 지점에 아이템을 추가하거나 삭제하는 작업은 O(1) 걸림
>- 연결 구조를 통해 물미 메모리를 연속적으로 사용하지 않아도 되서 관리가 쉬움
>- 데이터를 구조체로 묶어서 포인터로 연결한다는 개념은 다양한 활용이 가능함

---
# 8.1 팰린드롬 연결 리스트
**리트코드 234. Palindrome Linked List**  
**연결 리스트가 팰린드롬 구조인지 판별하라**

### 예제 1
- 입력
```{python}
1->2
```
- 출력
```{python}
false
```

### 예제 2
- 입력
```{python}
1->2->2->1
```
- 출력
```{python}
true
```

---
## 풀이 1. 리스트 변환
팰린드롬 여부를 판별하기 위해서는 **앞 뒤로 모두 추출할 수 있는 자료구조가 필요함**  
일반적인 스택 자료구조는 마지막 요소만 추출하는 연산만 있음  
하지만 파이썬의 리스트는 **pop()** 함수에 인덱스를 지정하여 원하는 위치를 자유롭게 추출할 수 있음  
> 따라서 이 문제는 연결 리스트를 **파이썬의 리스트로 변환하여 리스트의 기능을 이용하면** 쉽게 풀이할 수 있음

In [5]:
class ListNode(object):
    def __init__(self, val=0, next_val=None):
        self.val = val
        self.next_val = next_val

문제에서 기본으로 제공되는 **ListNode** 클래스

In [6]:
def is_palindrome(head: ListNode) -> bool:
    q: List = []
        
    if not head:
        return True
    
    node = head
    while node is not None:
        q.append(node.val)
        node = node.next_val
    
    while len(q) > 1:
        if q.pop(0) != q.pop():
            return False
    
    return True

In [8]:
input_list = ListNode(1)
input_list.next_val = ListNode(2)
input_list.next_val.next_val = ListNode(2)
input_list.next_val.next_val.next_val = ListNode(1)

is_palindrome(input_list)

True

---
## 풀이 2. 데크를 이용한 최적화
리스트가 아닌 **데크**를 이용하면 좀 더 최적화할 수 있음  
위 풀이에서  
```{python}
if q.pop(0) != q.pop():
```
부분을 보면 **q.pop(0)** 에서 모든 값이 한 칸씩 시프팅되며 시간 복잡도 O(n)이 발생함  
이 때문에 최적화를 위해서는 맨 앞에 데이터를 가져올 때 O(n) 이내에 처리할 수 있는 자료형이 필요함  
파이썬의 **데크(deque)** 는 이중 연결 리스트 구조로 양쪽 방향 모두 추출하는데 시간 복잡도 O(1)에 실행됨  
>만약 리스트로 처리했을 때 타임아웃이 발생하거나 오프라인 코딩 인터뷰에서 면접관이 이것이 효율적인가를 질문할 때, **양방향 모두 O(1)에 가능한 데크를 설명한 다음 이 자료형을 적용할 것이라고 얘기해볼 수 있음**
>- 파이썬에서 리스트를 데크로 수정하려면 딱 두 군데만 수정하면 됨
    >- []를 collection.deque로
    >- q.pop(0)을 q.popleft()로

In [11]:
import collections

def is_palindrome(head: ListNode) -> bool:
    q: Deque = collections.deque()
        
    if not head:
        return True
    
    node = head
    while node is not None:
        q.append(node.val)
        node = node.next_val
    
    while len(q) > 1:
        if q.popleft() != q.pop():
            return False
    
    return True

In [10]:
input_list = ListNode(1)
input_list.next_val = ListNode(2)
input_list.next_val.next_val = ListNode(2)
input_list.next_val.next_val.next_val = ListNode(1)

is_palindrome(input_list)

True

---
## 풀이 3. 런너를 이용한 풀이
사실 이 팰린드롬 연결 리스트 문제의 제대로 된 풀이는 **런너(Runner) 기법을 활용하는 것** 이라고 함  
> 빠른 런너(Fast Runner)와 느린 런너(Slow Runner)를 각각 출발 시킴
>- 팰린드롬이라면 **빠른 런너가 끝에 다다를 때 느린 런너는 정확히 중간 지점에 도달할 것**
>- 느린 런너는 중간까지 이동하면서 역순으로 연결 리스트 **rev**를 만들어 나감
>- 중간에 도달한 느린 런너는 나머지 경로를 이동할 때, 역순으로 만든 연결 리스트의 값들과 일치하는지 확인하면서 나가면 됨
>- 빠른 런너와 느린 런너 모두 초깃값은 head로 시작

In [21]:
def is_palindrome(head: ListNode) -> bool:
    
    rev = None
    slow = fast = head
    
    while fast and fast.next_val:
        fast = fast.next_val.next_val
        rev, rev.next_val, slow = slow, rev, slow.next_val
    
    if fast:
        slow = slow.next_val
    
    while rev and rev.val == slow.val:
        slow, rev = slow.next_val, rev.next_val
        
    return not rev

In [22]:
input_list = ListNode(1)
input_list.next_val = ListNode(2)
input_list.next_val.next_val = ListNode(2)
input_list.next_val.next_val.next_val = ListNode(1)

is_palindrome(input_list)

True

### 런너 기법
런너는 연결 리스트를 순회할 때 2개의 포인터를 동시에 사용하는 기법임
> **한 포인터가 다른 포인터보다 앞서게 하여 병합 지점이나 중간 위치, 길이 등을 판별할 때 유용**
>- 두 개의 포인터는 각각 빠른 런너, 느린 런너라고 부름
>- 대개 빠른 런너는 두 칸씩 건너 뛰고, 느린 런너는 한 칸씩 이동함
>- 이때 빠른 런너가 연결 리스트의 끝에 도달하면, 느린 런너는 정확히 연결 리스트의 중간 지점을 가리키게 됨
>- 이 같은 방식으로 중간 위치를 찾아내면, 여기서부터 값을 비교하거나 뒤집기를 시도하는 등 여려 활용이 가능
>- **연결 리스트 문제에서는 반드시 쓰이는 기법**